# The Greeks and Credit Spreads

The greeks are the partial-derivatives of the Black-Scholes equation with respect to each variable. So we are going to start with some theory and then we will code the greeks together with the credit spreads.

### Black–Scholes formula



For pricing a Put-Option:

 $$\textbf{P} = Ke^{−r(T−t)}N(−d_2)−SN(−d_1)$$
 
For pricing a Call-Option:

$$\textbf{C} = SN(d_1) - Ke^{-rt}N(d_2)$$


##### with the assumptions:


1. European options can only be exercised at expiration


2. No dividends are paid during the option's life


3. Market movements cannot be predicted


4. The risk-free rate and volatility are constant


5. Follows a lognormal distribution


##### with 

$S$ the spot price of the asset at time t


$T$ the maturity of the option. Time to maturity is defined as T−t


$K$ strike price of the option


$r$ the risk-free interest rate, assumed to be constant between t and T


$\sigma$, volatility of underlying asset, the standard deviation of the asset returns

## 1. The Greeks

The greeks are the partial-derivatives of the Black-Scholes equation with respect to each variable.

$\Delta$ — Delta is the first partial-derivative with respect to the underlying asset.
 
$\Gamma$ — Gamma is the 2nd partial-derivative with respect to the underlying asset.
    
$V$ — Vega is the partial-derivative with respect to volatility. 

$\Theta$ — Theta is the partial-derivative with respect to time until expiration and losing value per day.

$\rho$ — Rho is the partial derivative with respect to the given interest rate.

### 2. Credit Spread Ratios

$$Risk Ratio = \frac{Net \Theta} {\sqrt{Net \Delta^2 + Net \Gamma^2}}$$

$$Cost Ratio = \frac{Net Credit}{Share Cost + Margin Required}$$

In [ ]:
import datetime
import re
import pandas as pd
import QuantLib as ql
from pandas_datareader.data import Options
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import numpy as np
import yfinance as yf
from yahoo_fin.stock_info import get_quote_table 

In [ ]:
ticker = "AAPL"
expiration = datetime.date(2021, 2, 19)
info = get_quote_table(ticker)
current_price = info["Quote Price"]
yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
try:
    dividend_yield = float(
        yield_re.search(info["Forward Dividend & Yield"])["value"]
    )
except (KeyError, ValueError, TypeError):
    dividend_yield = 0.0# Fetch call option chain prices
chain=opt_sym.option_chain(opt_sym.options[1])    
calls=chain.calls
#calls = get_calls(ticker)# Setup instruments for Black-Scholes pricing
today = ql.Date.todaysDate()
underlying = ql.SimpleQuote(current_price)
exercise = ql.AmericanExercise(
    today,
    ql.Date(expiration.day, expiration.month, expiration.year)
)
dividendYield = ql.FlatForward(
    today, dividend_yield, ql.Actual360()
)
riskFreeRate = ql.FlatForward(today, 0.0008913, ql.Actual360())

def create_option(row):
    volatility = ql.BlackConstantVol(
        today,
        ql.UnitedStates(),
        row["volatility"],
        ql.Business252()
    )
    option = ql.VanillaOption(
        ql.PlainVanillaPayoff(ql.Option.Call, row["strike"]),
        exercise
    )
    process = ql.BlackScholesMertonProcess(
        ql.QuoteHandle(underlying),
        ql.YieldTermStructureHandle(dividendYield),
        ql.YieldTermStructureHandle(riskFreeRate),
        ql.BlackVolTermStructureHandle(volatility),
    )
    # Don't use the quoted implied vol
    # Calculate it out from the last price
    imp_vol = option.impliedVolatility(row["lastPrice"], process)
    implied_volatility = ql.BlackConstantVol(
        today,
        ql.UnitedStates(),
        imp_vol,
        ql.Business252()
    )
    process = ql.BlackScholesMertonProcess(
        ql.QuoteHandle(underlying),
        ql.YieldTermStructureHandle(dividendYield),
        ql.YieldTermStructureHandle(riskFreeRate),
        ql.BlackVolTermStructureHandle(implied_volatility),
    )
    option.setPricingEngine(
        ql.FdBlackScholesVanillaEngine(process, 1000, 1000)
    )
    return {
        "Name": row["contractSymbol"],
        "Strike": row["strike"],
        "Last": row["lastPrice"],
        "Bid": row["bid"],
        "Ask": row["ask"],
        "NPV": option.NPV(),
        "Delta": option.delta(),
        "Gamma": option.gamma(),
        "Theta": option.theta() / 365,
        "Volatility": imp_vol * 100,
    }# Filter down to only OTM strikes